
<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900">**5:** 多模态架构与融合技术</font>

在这个 notebook 中，我们将探讨多模态模型，重点关注如何将来自不同数据类型的信息融合，比如文本、图像和音频。我们将基于之前 notebook 讨论的原则，深入高级的主题，包括双编码器、早期融合（early-fusion）解码器、交叉注意机制和扩散模型（diffusion model）。

### **学习目标:**
- 理解**如何将不同模态编码**为紧凑的表示，捕捉局部和全局信息。
- 了解**联合优化**（joint optimization）在像 CLIP 这样的晚期融合（late-fusion）模型中的应用，这些模型独立处理文本和图像，并将它们投射到共享的嵌入空间。
- 认识到需要 **条件领域特定的解码器** 来生成复杂的输出，如音频和图像引导的响应。
- 探索 **文本引导的扩散模型的公式**，涉及一个非自回归解码器，能够生成复杂的输出，同时仍允许多模态条件。

到这个笔记本结束时，您将对这些多模态融合技术及其在现代 GenAI 应用中的相关性有深入的理解。

<hr>
<br>

## **5.1:** 定义模态？

在深入多模态架构之前，我们先来澄清一下模态的含义。

**模态**是一种特定形式或类型的数据，其特征在于其结构和传递信息的方式。常见的模态包括：
- **文本：**由单词或 token 组成的序列。
- **图像：**二维像素值数组。
- **音频：**表示声波的时间序列数据。
- **视频：**随时间变化的一系列图像。
- **传感器数据：** 来自加速度计或温度传感器等设备的测量。

在您之前的学习中，您接触到卷积作为一种稀疏推理机制，擅长将空间局部性融入模型预测中。这种方法在处理图像数据时表现出色，允许我们通过特征提取将像素转换为有意义的特征，然后可以用这些特征来条件化稠密网络，以充当分类器。

同样，每种数据类型（或模态）都有固有的关系，决定了其信息片段如何相互交互，而有效处理这些关系需要不同的架构。例如，我们一直关注 transformer，因为它们的 token 级稠密层和高效的注意机制，使其成为语言的优秀推理器。这是因为语言本质上是有序的序列，而注意机制特别适合对序列进行推理。但要记住，transformer 的注意力机制并不限于处理单词和句子——它足够灵活，可以应用于多种模态，并可以根据需要与其它结构的推理模块结合使用。

> <div><img src="imgs/multimodal.png" width="800"/></div>
> 
> **来源: [NVIDIA 在 SIGGRAPH 2023 的主题演讲 | NVIDIA](https://www.nvidia.com/en-us/events/siggraph/)**

**在这个 notebook 中，我们主要处理以下模态：**
- **自然语言：** 结构为有序的 token 序列，每个 token 在上下文中都携带语义。
- **图像：** 由像素值组成，可以在二维中捕捉微观（细节）和宏观（全局）方面。
- **音频：** 结构为时间序列数据集，其中每个点对应于特定的传感器读数或声波强度。

尽管每种模态都有其独特的挑战和架构，但在多模态系统中使用时，它们有一个共同的目标：***传递信息并帮助将其转换为可用的表示（无论是显式的还是隐式的）。*** 在这个 notebook 中，我们将培养对不同模态如何相互作用的高层次直觉，您也将有机会亲自进行实验。

<hr>
<br>

## **5.2:** 编码不同的模态

在之前的部分，我们花了相当多的时间讨论 transformer 如何编码和处理文本序列。正如您所见，transformer 在捕捉序列中 token 之间的关系方面表现出色，使其在文本分类、生成和翻译等各种任务中非常有效。文本的成功自然引导我们探索 transformer 是否可以处理其他具有自身结构和模式的数据类型或模态。

我们现在将逐步了解 transformer 如何编码和处理不同的模态，如音频和图像，每种模态都带来了独特的挑战，同时仍然受益于相同的基础 transformer 架构。

#### **文本嵌入**

到目前为止，您应该已经相当熟悉使用 transformer 进行文本嵌入。下面是一些来自之前 notebook 的代码，用于编码几条文本字符串，这些字符串稍后将用作实验样本：

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

text_captions = ("Cat with paint", "rock statue", "frogs on leaf", "jellyfish")
text_dialogue = ("Cats don't usually", 'like water', 'but this one likes paint', 'quite a lot')

# Load model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
inputs = tokenizer(text_captions + text_dialogue, padding=True, return_tensors=model.framework)
num_values = inputs.get("attention_mask").sum(axis=1)

# Get text embeddings
with torch.no_grad():
    outputs = model(**inputs)
text_embeddings = outputs.last_hidden_state  # (batch_size, sequence_length, hidden_size)
print(f"{text_embeddings.shape = }")

captions_embeddings = text_embeddings[:4]
dialogue_embeddings = text_embeddings[4:]

我们现在有八个嵌入序列，每个对应于输入中的一条文本字符串。序列中的每个 token 都有一个 768 维的嵌入，捕捉上下文和语义信息。

#### **音频嵌入**

音频实际上是有序的，类似于文本，不同的是它明确建模了时间，每个音频单位占用一个固定的时间间隔。处理音频的 transformer 通常在 Mel 频谱图上操作（声音的时频表示），可以与图像数据类似对待，每帧表示沿一个轴（通常是 x 轴）的一段时间，而另一轴（通常是 y 轴）则是采样频率。

> <div><img src="imgs/wav2vec2.png" width="600"/></div>
>
> **来源: [wav2vec 2.0: A Framework for Self-Supervised Learning of Speech Representations  (2202)](https://arxiv.org/abs/2006.11477)**

使用相同的 huggingface 处理逻辑，我们可以引入一个音频模型，比如 [**FAIR 的 wav2vec2 模型**](https://huggingface.co/docs/transformers/en/model_doc/wav2vec2)来处理我们的音频样本，读取我们之前的文本样本：

In [ ]:
# import IPython
# IPython.display.Audio("audio-files/paint-cat.wav")

In [ ]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2Model, AutoProcessor
import torchaudio
import numpy as np

# Load pre-trained tokenizer and model for Wav2Vec2 (a speech transformer)
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

# Load and preprocess audio (assuming the audio file is in .wav format)
waveform, sample_rate = torchaudio.load("audio-files/paint-cat.wav")  ## NOTE: Audio file is 4.005s ~ 4s
resampler = torchaudio.transforms.Resample(sample_rate, 16000, dtype=waveform.dtype)
waveform = resampler(waveform)[0]

inputs = processor(waveform, return_tensors=model.framework, sampling_rate=16000)

# Forward pass through the model to get audio features
with torch.no_grad():
    outputs = model(**inputs)
audio_hidden_states = outputs.last_hidden_state
print(audio_hidden_states.shape)  # Shape: (batch_size, sequence_length, hidden_size)

输出是音频的编码表示，可以用于下游任务。每个 768 维的向量表示音频文件的一小段（约 0.02 秒）的编码，通过一种编码器风格的 transformer 进行编码。

#### **图像？**
现在我们来看看图像，它们与文本和音频有着根本的不同。图像是像素值的 2D 数组，因此我们需要一种方法将它们转换为 transformer 可以处理的序列。[**视觉 transformer（ViT）范式**](https://huggingface.co/docs/transformers/en/model_doc/vit)就有了用武之地。

ViT 模型将图像视为一系列 patch。每个 patch 被映射到某种学习出来的向量表示（在当前的范式中通常会被压扁），这些向量被视为文本序列中的 token。Transformer 随后学习捕捉不同 patch 之间的关系，使其能够理解局部模式（patch 内部）和全局上下文（整个图像）。

> <div><img src="imgs/vit-model.png" width="800"/></div>
>
> **来源: [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale (2020)](https://arxiv.org/abs/2010.11929)**

让我们看看如何使用像 [**google/vit-base-patch16-224**](https://huggingface.co/google/vit-base-patch16-224) 这样的 ViT 启用编码器模型来编码一张图像：

In [ ]:
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import requests

# Load the pre-trained feature extractor and model
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224')

# Load an example image
img_files = ["paint-cat", "rock-head", "tree-frog", "two-jelly"]
images = [Image.open(f"img-files/{name}.jpg") for name in img_files]    

# Preprocess the image to fit model input
inputs = feature_extractor(images=images, return_tensors=model.framework)

# Forward pass through the model to get image features
with torch.no_grad():
    outputs = model(**inputs)
image_hidden_states = outputs.last_hidden_state
print(image_hidden_states.shape)  # Shape: (batch_size, num_patches, hidden_size)

在这种情况下，ViT 组件将图像分成多个 patch，通过 transformer 处理这些 patch，并输出编码后的 patch 序列。每个 patch 现在包含关于其邻近 patch 的上下文信息，使模型能够以结构化的方式理解图像。

<hr>
<br>

## **5.3：** 联合投影（Joint Projection）

我们现在知道，使用 transformer 来获取文本、音频和图像的语义密集嵌入是合理的。无论数据的底层结构如何，我们都能够提取出*某种*形式的表示，768 维的嵌入维度。尽管如此，我们还有一些差异需要解决：

- 语言嵌入仍然是一系列按 token 计算的嵌入。
- 音频嵌入是按时间窗口计算的嵌入。
- 图像嵌入是按 patch 计算的嵌入。

由于每个嵌入器中都有注意力机制，且每个 token 对于序列都有全局上下文。理论上，我们*应该*优先使用提供的 `[CLS]` token，因为它在训练数据中的持续存在，但在这个例子中我们将忽略它，仅仅对嵌入向量求平均。如果您好奇为什么，(A) 这些没有进行联合优化，会给出更不稳定的相似性结果，而不是相关值的平均，(B) 这些音频嵌入在训练期间没有包含 `[CLS]` token。

让我们提取每种模态的平均嵌入，以获得每个输入的单一向量表示：

In [ ]:
## Eliminate contribution from pad tokens, since these values were obtained via batching
captions_avg_embeds = torch.sum(captions_embeddings[:,1:,:], axis=1).detach() / (num_values[:4].view(-1, 1) - 1)
dialogue_avg_embeds = torch.sum(dialogue_embeddings[:,1:,:], axis=1).detach() / (num_values[4:].view(-1, 1) - 1)

## Remove CLS token from ViT, since we said we just wanted embedding average
image_avg_embeds = torch.mean(image_hidden_states.view(4, -1, 768)[:,1:,:], axis=1).detach()

## Wav2Vec2 does not have CLS token, but we still want to divide our 4s clip into four 1s clips.
audio_avg_embeds = torch.mean(audio_hidden_states.view(4, -1, 768), axis=1).detach()

print(f"{captions_avg_embeds.shape = }")
print(f"{dialogue_avg_embeds.shape = }")
print(f"{image_avg_embeds.shape = }")
print(f"{audio_avg_embeds.shape = }")

**现在每个输入在所有模态中都有一个唯一的 768 维嵌入。** 然而，这些嵌入来自独立训练的模型，且在任何共享空间中并未对齐。为了说明这一点，让我们比较不同模态嵌入之间的相似性：

In [ ]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt

def plot_similarity(similarity, xlab, ylab, xticks, yticks, ax):
    # Visualization of the similarity matrix on the provided axis
    sns.heatmap(similarity.numpy(), annot=True, cmap='coolwarm', xticklabels=xticks, yticklabels=yticks, ax=ax)
    ax.set_title(f"Similarity between {xlab} and {ylab} Embeddings")
    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)

fig, axs = plt.subplots(2, 3, figsize=(18, 12))

## Expected Heatmap For Top Row: Diagonal Matrix. Plot [0][0] is a demo of this
sim_mtx = (image_avg_embeds @ image_avg_embeds.T).softmax(dim=0)
plot_similarity(sim_mtx, "Image", "Image", img_files, img_files, axs[0][0])

sim_mtx = (captions_avg_embeds @ image_avg_embeds.T).softmax(dim=0)
plot_similarity(sim_mtx, "Captions", "Image", text_captions, img_files, axs[0][1])

sim_mtx = (audio_avg_embeds @ dialogue_avg_embeds.T).softmax(dim=0)
plot_similarity(sim_mtx, "Audio", "Dialogue", text_dialogue, text_dialogue, axs[0][2])

## Expected Heatmap For Bottom Row: Undefined, but [1][2] doesn't look terrible...
sim_mtx = (dialogue_avg_embeds @ captions_avg_embeds.T).softmax(dim=0)
plot_similarity(sim_mtx, "Dialogue", "Captions", text_dialogue, text_captions, axs[1][0])

sim_mtx = (dialogue_avg_embeds @ image_avg_embeds.T).softmax(dim=0)
plot_similarity(sim_mtx, "Dialogue", "Image", text_dialogue, img_files, axs[1][1])

sim_mtx = (audio_avg_embeds @ image_avg_embeds.T).softmax(dim=0)
plot_similarity(sim_mtx, "Audio", "Image", text_dialogue, img_files, axs[1][2])

plt.tight_layout()
plt.show()

如果您仔细观察，就会发现仍然存在一些合理的关系。一些输入嵌入与某些输出嵌入之间的强对齐可能表明存在某种协同（synergy）效应（可能在右下角），但也可能完全没有意义（比如左下角）。这是因为**这些表示是独立优化用于下游学习的**，因此会形成完全不同的隐式表示。

### **用 CLIP 进行联合优化**

为了应对这个限制，[**CLIP（对比语言-图像预训练）**](https://openai.com/index/clip/)模型专门训练以将不同模态（如图像和文本）投影到一个共享的嵌入空间，在这个空间中它们可以轻松进行比较。这个共享空间在两个模态之间进行联合优化，以便模型能学到一致的表示，即相似的图像和文本对被映射得更近，无关的被推得更远。

CLIP 通过**对比学习目标**实现这一点，它通过最小化成对图像及其标题在共享空间中的距离，同时最大化无关对之间的距离，来对齐嵌入。这个联合优化过程促使模型学习有用的模块化表示，以便于图像-文本检索等多模态任务。

> <div><img src="imgs/clip-arch.png" width="1000"/></div>
>
> **来源: [Learning Transferable Visual Models From Natural Language Supervision  (2021)](https://arxiv.org/abs/2103.00020)**

为了展示联合优化的力量，让我们探索一下 CLIP 嵌入在它实际设计的任务上的表现——将图像与其对应的文本描述对齐。这个任务是 CLIP 的核心优势之一，我们将看看它在共享嵌入空间中处理图像和标题对齐的能力有多好。

In [ ]:
from transformers import CLIPProcessor, CLIPModel

text_captions = ("Cat with paint", "rock statue", "frogs on leaf", "jellyfish")
text_dialogue = ("Cats don't usually", 'like water', 'but this one likes paint', 'quite a lot')

#######################################################################
## TODO: Load in a clip model of choice, reading over the model-card recommendations.
## HINT: We'd recommend openai/clip-vit-base-patch32
model = None
processor = None

#######################################################################
## TODO: Compute the text and image embeddings for analysis
inputs_text = None
inputs_images = None

#######################################################################
## TODO: Get the text and image embeddings for final visual
with torch.no_grad():
    text_embeddings = None
    image_embeddings = None

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

sim_mtx = (text_embeddings[:4] @ image_embeddings.T).softmax(dim=0)
plot_similarity(sim_mtx, "Captions", "Image", text_captions, img_files, axs[0])

sim_mtx = (text_embeddings[4:] @ image_embeddings.T).softmax(dim=0)
plot_similarity(sim_mtx, "Dialogue", "Image", text_dialogue, img_files, axs[1])

plt.tight_layout()
plt.show()

这表明训练期间的联合优化可以强制模态之间的关系，从而导致有用的涌现行为。通过 CLIP，嵌入在模态间对齐，相似性矩阵显示出其将图像与标题对齐的有意义关联。请注意，这并不一定使它们在*任意任务*上对齐，正如对话-图像相似性结果所展示的那样，可能有点违反直觉。

<details>
<summary><b>练习参考答案</b></summary>

```python
#######################################################################
## TODO: Load in a clip model of choice, reading over the model-card recommendations.
## HINT: We'd recommend openai/clip-vit-base-patch32
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

#######################################################################
## TODO: Compute the text and image embeddings for analysis
inputs_text = processor(text=text_captions + text_dialogue, return_tensors="pt", padding=True)
inputs_images = processor(images=images, return_tensors="pt", padding=True)

#######################################################################
## TODO: Get the text and image embeddings for final visual
with torch.no_grad():
    text_embeddings = model.get_text_features(**inputs_text)
    image_embeddings = model.get_image_features(**inputs_images)

```
</details>

<hr>
<br>

## **5.4：** 将多模态编码器与解码器结合

到目前为止，我们已经看到将不同模态编码为紧凑表示形式，使我们能够比较和对比它们以用于下游任务。CLIP 使用的策略允许我们独立计算每种模态的嵌入，使其在将图像与标题对齐或根据相似性检索相关信息等任务中变得有用。这被称为**双编码器公式**，并利用了**后融合**，其中来自两种模态的信息在推理过程中融合较晚。

然而，仅在最后比较嵌入对于更复杂的任务和更深层次的推理来说还不够。为了解决这个问题，一些模型选择采用**早期融合**方法，比如**交叉编码**，将两个分布从一开始就通过相同的输入路径传入。这类似于 BERT 的下一个句子预测任务，但当两个序列实际上来自不同领域时，该模型包含一个将数据投影到共享或共同合理输入空间的实用工具。

### **交叉注意力：在生成任务中桥接模态**

虽然早期融合对于更复杂的任务确实很重要，但编码再次不足以生成复杂的结构化数据输出。例如，从图像生成文本描述（图像描述）或将音频转录为文本（语音识别）这样的任务不应该仅仅依靠编码器方法。我们真正需要的是一个领域特定的解码器，它可以解释编码的信息，并在关于领域结构的合理先验假设下生成有意义的输出。通过使用解码器风格的 transformer 架构生成基于文本条件的序列，我们可以对非文本嵌入进行条件处理和/或生成非文本序列。

#### **使用 ViT + GPT2 进行图像描述**

在图像描述中，我们的目标是生成与给定图像对应的描述性句子。这需要：
- 一个视觉编码器（例如 ViT）来从图像中提取特征
- 一种语言解码器（例如 GPT-2）来生成文本。

交叉注意力机制通过允许解码器专注于单词创建，同时让编码器专注于其视觉输入。我们可以通过相对轻量级的 [**vit-gpt2-image-captioning 模型**](https://huggingface.co/nlpconnect/vit-gpt2-image-captioning)来展示这一点，该模型在[**作者的博客文章中**](https://ankur3107.github.io/blogs/the-illustrated-image-captioning-using-transformers/)有更详细的描述。

让我们通过使用 Huggingface pipeline 和手动推理过程来看一个示例，以验证我们底层输入路径的直觉是否合理：

In [ ]:
import IPython
from transformers import pipeline
from PIL import Image

#################################################################################################
## Easy Pipeline

img_files = ["paint-cat", "rock-head", "tree-frog", "two-jelly"]
images = [Image.open(f"img-files/{name}.jpg") for name in img_files]    

vit_pipe = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
for name, image in zip(img_files, images):
    print(f"{name}: {vit_pipe(image)}")
print()

#################################################################################################
## Manually

## Generate the inputs for both the image encoder AND the text decoder
## - We want to generate text, so the decoder should deal with it to avoid structure loss
## - We want to image to HELP the generation but do not want image structure to propagate to output
starter_phrase = "I love"
starter_tokens = vit_pipe.tokenizer([starter_phrase] * len(images), return_tensors=vit_pipe.framework)
image_embeddings = vit_pipe.image_processor(images, return_tensors=vit_pipe.framework)
print("Start Inputs:", {k:getattr(v, "shape", v) for k,v in starter_tokens.items()})
print("Image Inputs:", {k:getattr(v, "shape", v) for k,v in image_embeddings.items()})

# Generate transcription by passing all of this into the model. You'll get a vector of token IDs
generated_ids = vit_pipe.model.generate(**starter_tokens, **image_embeddings)
print("\nModel Outputs:", generated_ids.shape)

## Detokenize back to actual human language and print!
transcription = vit_pipe.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print("\nTranscript:", transcription)

<br>

#### **使用 Whisper 进行语音识别**
在语音识别中，目标是从音频输入生成文本转录。就像图像描述一样，这需要一个编码器来处理音频和一个解码器来生成文本。在这种情况下，像 [**OpenAI Whisper**](https://openai.com/index/whisper/) 这样的模型通过交叉注意力，在生成转录的每个单词时关注音频输入的不同部分，学习将原始音频信号映射到文本。

> <div><img src="imgs/whisper-arch.png" width="700"/></div>
>
> **来源: [Introducing Whisper | OpenAI](https://openai.com/research/whisper)**

让我们看看 Whisper 如何用于语音转录，并注意代码与 ViT 示例的不同之处（以及为什么结合早期的 Wav2Vec2 音频编码器是合理的）：

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import torchaudio

from glob import glob

dev = "cuda" if torch.cuda.is_available() else "cpu"

## whisper-large-v2 recommended, but slow. For only english, whisper-base is fine
## Upgrading to a larger model will lead to better hard-case results and formatting
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3", device=dev)
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3").to(dev)

# Load and process audio file
def transcribe_file(audio_file):
    audio, _ = torchaudio.load(audio_file)
    audio = audio.squeeze().numpy()
    inputs = processor(audio, return_tensors=model.framework, sampling_rate=16000, device=dev)
    generated_ids = model.generate(inputs["input_features"].to(dev), max_length=50)
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
    return transcription[0]

for filename in sorted(glob("audio-files/*")):
    print(f"{filename}: {transcribe_file(filename)}")

<br>

#### **[附注] 仅解码器的多模态模型**

我们之前讨论了交叉注意力的优缺点，可以再次得到使用轻量级模型的结论。然而，仅解码器的扩展也非常流行，在**视觉-语言模型**（VLM）也存在。在下一个 notebook 中会继续探索，但高层次的直觉非常简单：

- **线性投影**将图像特征映射到与 LLM 的文本嵌入对齐的空间。从视觉输入的角度来看，这实际上是一个多模态编码练习，但从语言的角度来看，它是仅解码器。
- **解码器**（通常是基于 transformer 的 LLM）将投影的图像和文本一起处理，有效地通过语言模型的解码过程执行多模态推理。

这使得在上下文中进行图像推理甚至密集描述（多图像/视频推理）变得非常自然，但这要求模型学习如何处理 token 以及一些密集的图像投影，因此训练会变得更复杂。有关这些模型的更多细节，请关注下一个 notebook 中基于 VLM 的练习，并考虑与 [**LLaVA (2023)**](https://llava-vl.github.io)、[**Llama 3.2 (2024)**](https://ai.meta.com/blog/llama-3-2-connect-2024-vision-edge-mobile-devices/)、[**NVLM (2024)**](https://arxiv.org/abs/2409.11402) 以及其它先进的视觉-语言模型进行互动。

<hr>
<br>

## **5.5：扩散解码器简介**


对于那些 transform 或自回归范式本身不足以生成高质量结果的输出领域（即，不是自然序列的表示），我们可能需要一些更适合您输出模态结构的专用解码器。其中一个特别值得关注的方法是逐步去噪，目标是逐步传递信息到最终输出，而不是一次给出一个完成的构建块。为了说明这一点，我们可以引入策略网络（policy network）的例子！

#### **策略网络**

**策略网络**，通常用于强化学习，旨在生成将系统从一个状态转移到另一个状态的动作。这意味着它们大致学习一个 `predict_action(past_actions, past_observations)` 函数，该函数生成一个好的 `new_action`，以生成 `new_observation = act_in_environment(new_action)`。 

您可能已经猜到 `predict_action` 可以是一个简单的动作或状态转移系统，预测以下之一：
$$P(A_t \ | \ O_{t-1}) \text{ such that } O_t = E(A_{\le t}) \text{ is desirable}$$
$$P(\Delta A_t \ | \ A_{t-1}, O_{t-1}) \text{ where } A_t = A_{t-1} + \Delta A_t \text{ such that } O_t \text{ is desirable}$$

换句话说，您绝对可以学习一个转移函数来预测一个新的动作或配置（configuration） $A'$，或者一个配置增量 $\Delta A'$，以从一个状态转移到另一个状态，所有这些都是基于： 
- **过去的状态**，或者您可以直接影响的真实自由度
- **计算出来的观察**，或来自环境的奖励、图像嵌入、文本指令等，定义您的优化目标。 

老实说，这对于一些轻量级动态层还是非常有用的，目的在于在执行模糊定义的任务（即“向前移动”）和执行纠正措施的过程中将某些东西从一个点移动到另一个点。 

本课程的这一阶段，您可能会发现还可以用 transform 架构。也许下面这样的公式会有效：

$$P(A_t \ | \ O_{<t}, A_{<t}) \text{ such that } O_t \text{ is desirable}$$
$$\text{AKA Predict each next action autoregressively from previous actions and observations}$$

这将引导您走向 [**Decision Transformers (2021)**](https://proceedings.neurips.cc/paper_files/paper/2021/file/7f489f642a0ddb10272b5c31057f0663-Paper.pdf) 和 [**VIMA (2023)**](https://vimalabs.github.io/) 等模型，这些模型在可泛化的序列条件动作模型方面表现良好。您会发现，擅长序列推理的架构可以利用数据的序列结构，生成洞察力和新数据。

#### **使用扩散进行序列生成**

还有其它的生成架构，它们对数据的序列特性不太感兴趣，更关注将数据从初始表示转为最终表示的“进程”。即使是像机器人向目标推进这种可自回归建模的自然任务序列，也可以通过以下公式用**扩散**进行建模：

$$P(\Delta A^{k}_{1..t} \ | \ A^{k}_{1..t}, O_{0}, k) \text{ such that a finally-denoised sequence } A^0_{1..t} = A^K_{1..t} + \Delta A^{K}_{1..t} + ... + \Delta A^{1}_{1..t} \text{ is desirable}$$
$$\text{ and } A^K_{1..t} \text{ is an initial pointset which is in the same dimension as - and can be deformed into - the sequence } A^0_{1..t}$$

在这种方法中（虽然简化了，但仍能传达要点），模型是这样生成其最终输出的：
- **逐步和迭代地**，在多个步骤中精炼中间阶段，而不是直接自回归到最终输出。
- **作为状态空间中的一系列迭代变换**，而不是严格的顺序过程。
- **利用随机原则**，通过一系列去噪步骤从初始噪声样本去噪数据点。 

这使得扩散模型能够捕捉复杂的依赖关系，并逐步提高生成效果，这在图像生成等任务中非常有效，因为数据结构从迭代改进中受益，而不是逐步预测。

> <div><img src="imgs/policy-methods.png" width="800"/></div>
>
> 图片来源于 **[Diffusion Policy: Visuomotor Policy Learning via Action Diffusion (2024)](https://diffusion-policy.cs.columbia.edu/)**

---
<details>

<br>
    
<summary><strong>扩散策略详情：</strong></summary>

上述提到的<a href="https://diffusion-policy.cs.columbia.edu/" target="_blank"><strong>扩散策略</strong></a>方法的实际提议架构结合了 transform 和/或卷积以进行窗口化的顺序推理。它们的公式使用梯度 $\nabla$ 而不是增量 $\Delta$，但用我们之前的公式重新表述大致相当于： 
$$P(\nabla A^{k}_{b..t} \ | \ A^{k}_{b..t}, O_{a..<b}, k) \text{ with the denoising } A^{k-1}_{b..t} = \alpha(A^k_{b..t} - \gamma \nabla A^k_{b..t} + \mathcal{N}_{b..t}(0, \sigma^2I)) \text{ performed with sampling parameters }(\alpha, \gamma, \sigma)$$

有效地，$\nabla A^{k}_{b..t}$ 是学习到的 $k$-方差噪声，使得 $A^{k}_{b..t}$ 从去噪序列 $A^0_{b..t}$ 偏移，基于其条件观察 $O_{a..b}$，并以此目标进行训练。

> <div><img src="imgs/diffusion_policy.png" width="1000"/></div>
>
> **来源: [Diffusion Policy: Visuomotor Policy Learning via Action Diffusion (2024)](https://diffusion-policy.cs.columbia.edu/)**
<hr>

有关扩散模型及其特定假设和训练公式的更多信息，我们推荐[**构建基于扩散模型的生成式 AI 应用**](https://www.nvidia.cn/training/instructor-led-workshops/generative-ai-with-diffusion-models/)。


</details>

---

<br>

## **5.6：文本引导的图像扩散**

现在我们有两种不同的方法来逐步生成某种模态，**自回归**和**迭代优化**（iteratively-refine）。我们之前了解到自回归在序列输出方面表现出色，但现在提出了动作策略预测作为一个有争议的例子，这取决于所需的结果和推理方式。

图像同样是个有争议的问题；虽然视觉 transform 在处理图像方面表现很好，但在生成图像时却不太行，因为没有机制来保障高频或细粒度的细节。后来发现，扩散模型在图像生成方面非常有用，因为它们能够从早期未优化的迭代中捕捉到高频噪声模式，并在去噪过程中将其塑造成有趣的高保真细节：

> <div><img src="imgs/diffusion_img.png" width="900"/></div>
>
> **来源: [什么是扩散模型？| Lil'Log](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/)**

那么我们如何将“扩散引导的图像生成”这个想法变成“文本引导的图像生成”？

- 假设我们有一个架构 - 可能是 [**U-Net**](https://arxiv.org/pdf/1505.04597) - 能够学习一个有效的去噪函数 $p_{\theta}(x_t | x_{t-1})$
- 假设我们有一个模型，能够将文本提示编码为一个语义密集的有用表示 $E$。
- 然后我们可以简单地将我们的去噪函数条件化为 $p_{\theta}(x_t | x_{t-1}, E)$

那么我们如何有效地用 $E$ 来条件化去噪器呢？其实，原作者在去噪器中引入了一个注意力机制，并且在自注意中保持了一些该机制的衍生形式。

> <div><img src="imgs/latent-diffusion.png" width="750"/></div>
>
> **来源: [High-Resolution Image Synthesis with Latent Diffusion Model (2022)](https://arxiv.org/abs/2112.10752)**

<br>

#### **引入扩散模型**

以下代码块允许您引入一种最先进的图像扩散模型，[**StabilityAI's Stable Diffusion XL (SDXL-1.0) model**](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0)。它与其它流行的基于扩散的图像生成器如 [**OpenAI 的 Dalle 模型**](https://openai.com/index/dall-e-3/) 并列，并通过简单的基于提示的 API 进行操作，如下所示：

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to("cuda")

prompt = "An astronaut riding a green horse"

images = pipe(prompt=prompt).images[0]
images.show()

从这个抽象的解析中，很难看到任何内部细节，确实有点像魔法。由于各种优化以及扩散模型内部过程的不稳定特性，支持这些的 [**`diffusers`**](https://huggingface.co/docs/diffusers/en/index) 库拥有更复杂和动态的工作流结构。尽管如此，我们仍然可以利用事件循环来展示在推理过程中图像是如何逐步被优化的。

本课程不涉及编码细节，但有几点我们可以注意到：

- 在接下来的演示中，由于上述动态的工作流结构，我们不得不回退到使用回调说明符（callback specifier）。这为我们提供了一个方便的窗口来查看内部事件循环，也适用于 `transformers` 库及许多其它库。尝试重建导致这个回调过程会更加复杂。

- 您可能会注意到，我们在最后仍然是通过某个 `vae.decode` 方法来解码最终输出的。VAE 代表[**变分自编码器**](https://en.wikipedia.org/wiki/Variational_autoencoder)，是一种将数据通过瓶颈（bottleneck）- 即自编码器，包含预瓶颈编码器和后瓶颈解码器 - 传递的结构，同时使用编码器预测均值和标准差，以便采样一个变体或随机值作为解码器输入（变分）。在这里，这个编码器和解码器的作用是在像素空间和一些紧凑表示之间进行映射，但去噪函数在“从随机噪声解码”的上下文中也充当了解码器。

In [ ]:
from diffusers import DiffusionPipeline
import torch
import matplotlib.pyplot as plt

def decode_latents(self, latents):
    ## Borrowed from source: https://github.com/huggingface/diffusers/blob/e2ead7cdcc00859533e6bec7b0707a6fb0efef0a/src/diffusers/pipelines/stable_diffusion_xl/pipeline_stable_diffusion_xl.py#L1252
    ## This is a class method of the pipeline, so just pass in the pipeline for `self` and it should largely work.
    
    ## VARIOUS PREP UTILITIES, INCLUDING MODEL CASTING AND NORMALIZATION
    needs_upcasting = self.vae.dtype == torch.float16 and self.vae.config.force_upcast
    if needs_upcasting:
        self.upcast_vae()
        latents = latents.to(next(iter(self.vae.post_quant_conv.parameters())).dtype)
    elif latents.dtype != self.vae.dtype and torch.backends.mps.is_available():
        self.vae = self.vae.to(latents.dtype)
    has_latents_mean = hasattr(self.vae.config, "latents_mean") and self.vae.config.latents_mean is not None
    has_latents_std = hasattr(self.vae.config, "latents_std") and self.vae.config.latents_std is not None
    if has_latents_mean and has_latents_std:
        latents_stat_fn = lambda conf_stat: torch.tensor(conf_stat).view(1, 4, 1, 1).to(latents.device, latents.dtype)
        latents_mean = latents_stat_fn(self.vae.config.latents_mean)
        latents_std = latents_stat_fn(self.vae.config.latents_std)
        latent_scale = latents_std / self.vae.config.scaling_factor + latents_mean
    else:
        latent_scale = 1 / self.vae.config.scaling_factor

    ## ACTUAL DECODING
    image = self.vae.decode(latents * latent_scale, return_dict=False)[0]

    ## CLEANUP AND RETURN
    if needs_upcasting:
        self.vae.to(dtype=torch.float16)
    return image

# Define a list to hold the images at each step
step_images = []
    
# Define a callback function to save images at each step
def save_step_image(pipeline, step, timestep, callback_kwargs):
    latents = callback_kwargs['latents']
    image = decode_latents(pipeline, latents)
    step_images.append(image)
    return callback_kwargs

# Set the prompt
prompt = "An astronaut riding a green horse"

# Run the pipeline with the callback function
pipe(
    prompt=prompt,
    num_inference_steps=10,  # Reduced for quicker visualization
    guidance_scale=7.5,
    callback_on_step_end=save_step_image,
    callback_on_step_end_tensor_inputs=['latents'],
    output_type='latent'  # So that we can decode latents ourselves
)

# Visualize the denoising steps
fig, axes = plt.subplots(1, len(step_images), figsize=(20, 5))
for i, img in enumerate(step_images):
    axes[i].imshow((img[0].permute(1, 2, 0).cpu().numpy() * 0.5 + 0.5).clip(0, 1))
    axes[i].set_title(f"Step {i+1}")
    axes[i].axis("off")

plt.tight_layout()
plt.show()

<hr>
<br>

# <font color="#76b900">**总结**</font>

在这个 notebook 中，我们探讨了不同的多模态模型如何处理文本、图像和音频相关的任务。我们讨论了双编码器融合（如 CLIP）、用于更深层次推理的交叉编码器，以及像交叉注意力和文本引导的扩散模型这样的高级技术。

### 关键点：

- 不论数据的模态如何，都可以努力将数据从一种结构转换为另一种，同时尽量保留后续处理所需的重要细节。
- 双编码器支持后期融合，模态被分别处理并在共享空间中进行比较（例如，CLIP）。
- 交叉编码器使用早期融合，将多个输入一起处理，以实现更丰富的交互。
- 交叉注意力允许解码器动态引用编码输入，这对图像描述和语音识别等任务至关重要。
- 与逐个 token 生成的自回归方法不同，扩散模型通过利用基于噪声的迭代优化方法生成高复杂度的输出。

---

### 下一步：
我们鼓励您探索更高级的多模态模型和扩散技术，以进一步加深对融合及其在有趣的生成式 AI 应用中的理解。在下一个 notebook 中，我们将探讨更多资源密集型的选项，并考虑如何在合理受限或可扩展的环境中使用、简化和部署它们。

In [ ]:
## Please Run When You're Done!
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)